<a href="https://colab.research.google.com/github/MiyokoPang/IR/blob/master/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Prerequisite

In [1]:
!git clone https://github.com/MiyokoPang/IR.git
!pip install pymysql

Cloning into 'IR'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 5), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 871.37 KiB | 25.63 MiB/s, done.
Resolving deltas: 100% (5/5), done.


# 3B. FYP_Deep_Learning_Models

In [7]:
# Cell 1: Import Libraries and Configuration
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from datetime import datetime
from tqdm.notebook import tqdm

# Database
from sqlalchemy import create_engine
import pymysql

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Conv1D, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")
print("Libraries imported.")
print(f"TensorFlow version: {tf.__version__}")

# Database Configuration
db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '',
    'database': 'trading_system'
}
db_url = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
engine = create_engine(db_url)

# Path Definition
base_dir = Path.cwd().parent
output_dir = base_dir / "content" / "IR" / "4_Results" / "Deep_Learning_Results"
output_dir.mkdir(exist_ok=True)

print(f"Results will be saved to: {output_dir}")

Libraries imported.
TensorFlow version: 2.19.0
Results will be saved to: /content/IR/4_Results/Deep_Learning_Results


In [8]:
# Cell 2: Load Sequence Data
print("Loading sequence data from database.")

try:
    df = pd.read_sql("SELECT * FROM data_daily_merged", con=engine)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by=['ticker', 'date'])

    print(f"Successfully loaded {len(df):,} rows.")
    print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
    print(f"Number of unique tickers: {df['ticker'].nunique()}")

    print("\nAvailable columns:")
    print(df.columns.tolist())

except Exception as e:
    print(f"ERROR loading data: {e}")
    raise

Loading sequence data from database.
ERROR loading data: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Cell 3: Define Feature Sets for Deep Learning
print("Defining feature sets for sequence models.")

# 1. Price_Only - Technical features
price_features = ['close', 'volume']

# 2. Price_Base_Sentiment - Price + basic sentiment scores
base_sentiment_features = price_features + [
    'textblob_polarity',
    'vader_compound',
    'finbert_compound'
]

# 3. Price_Full_Sentiment - Price + all sentiment features
full_sentiment_features = price_features + [
    'textblob_polarity',
    'vader_compound',
    'finbert_compound',
    'news_count',
    'has_news',
    'days_since_news',
    'sentiment_freshness',
    'textblob_polarity_volatility',
    'vader_compound_volatility',
    'finbert_compound_volatility'
]

feature_sets = {
    "Price_Only": price_features,
    "Price_Base_Sentiment": base_sentiment_features,
    "Price_Full_Sentiment": full_sentiment_features
}

print(f"Total feature sets: {len(feature_sets)}")
print(f"- Price_Only: {len(price_features)} features")
print(f"- Price_Base_Sentiment: {len(base_sentiment_features)} features")
print(f"- Price_Full_Sentiment: {len(full_sentiment_features)} features")

In [ ]:
# Cell 4: Sequence Generation Functions
def create_sequences(data, feature_cols, target_col='close', sequence_length=60):
    """
    Create sequences for time series prediction.
    """
    X, y = [], []

    for i in range(sequence_length, len(data)):
        X.append(data[feature_cols].iloc[i-sequence_length:i].values)
        current_close = data[target_col].iloc[i-1]
        next_close = data[target_col].iloc[i]
        return_val = (next_close - current_close) / current_close
        y.append(return_val)

    return np.array(X), np.array(y)

def prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=60, train_ratio=0.8):
    """
    Prepare train/test sequences for a single ticker.
    """
    ticker_df = df[df['ticker'] == ticker].copy()
    ticker_df = ticker_df.sort_values('date').reset_index(drop=True)

    if len(ticker_df) < sequence_length + 50:
        return None

    scaler = StandardScaler()
    ticker_df[feature_cols] = scaler.fit_transform(ticker_df[feature_cols])

    X, y = create_sequences(ticker_df, feature_cols, sequence_length=sequence_length)
    split_idx = int(len(X) * train_ratio)

    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    if len(X_test) < 5:
        return None

    return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

print("Sequence generation functions defined.")

In [ ]:
# Cell 5: Model Architecture Builders
def build_lstm_model(input_shape, units=50):
    model = Sequential([
        LSTM(units, input_shape=input_shape, return_sequences=False),
        Dropout(0.2),
        Dense(25, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def build_bilstm_model(input_shape, units=50):
    model = Sequential([
        Bidirectional(LSTM(units, return_sequences=False), input_shape=input_shape),
        Dropout(0.2),
        Dense(25, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def build_cnn1d_model(input_shape, filters=64):
    model = Sequential([
        Conv1D(filters=filters, kernel_size=3, activation='relu', input_shape=input_shape),
        Conv1D(filters=filters//2, kernel_size=3, activation='relu'),
        Dropout(0.2),
        keras.layers.Flatten(),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def build_gru_model(input_shape, units=50):
    model = Sequential([
        GRU(units, input_shape=input_shape, return_sequences=False),
        Dropout(0.2),
        Dense(25, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model_builders = {
    'LSTM': build_lstm_model,
    'BiLSTM': build_bilstm_model,
    'CNN1D': build_cnn1d_model,
    'GRU': build_gru_model
}

print("Model architecture builders defined.")
print(f"Available architectures: {list(model_builders.keys())}")

In [ ]:
# Cell 6: Helper Functions
def calculate_metrics(y_true, y_pred, model_name, ticker, feature_set_name):
    mask = ~(np.isnan(y_true) | np.isnan(y_pred) | np.isinf(y_true) | np.isinf(y_pred))
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]

    if len(y_true_clean) == 0:
        return None

    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    r2 = r2_score(y_true_clean, y_pred_clean)
    correct_direction = np.sign(y_true_clean) == np.sign(y_pred_clean)
    directional_accuracy = np.mean(correct_direction) * 100

    return {
        'ticker': ticker,
        'model': f"{model_name}_{feature_set_name}",
        'feature_set': feature_set_name,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'directional_accuracy': directional_accuracy
    }

print("Helper functions defined.")

In [ ]:
# Cell 7: Main Training Loop (OPTIMIZED - Prevents Retracing)
print("Starting Deep Learning Model Training Loop.")

all_results = []
sequence_length = 60
epochs = 50
batch_size = 32

# Get ticker list
ticker_list = sorted(df['ticker'].unique())
print(f"Total available tickers: {len(ticker_list)}")

# Setting ticker limit
ticker_list = ticker_list[:100]  # Reduced to 100 for faster testing
print(f"Training on: {len(ticker_list)} tickers.")

# Early stopping callback
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

# Prefilter tickers by data availability
print("\nPre-filtering tickers by data availability.")
valid_tickers = []
for ticker in tqdm(ticker_list, desc="Validating Tickers"):
    ticker_df = df[df['ticker'] == ticker]
    if len(ticker_df) >= sequence_length + 100:  # Ensure enough data
        valid_tickers.append(ticker)

print(f"Valid tickers after filtering: {len(valid_tickers)} (removed {len(ticker_list) - len(valid_tickers)} with insufficient data)")
ticker_list = valid_tickers

# Adjust model weights rather than rebuilding
print("\nPre-building model templates.")
model_templates = {}

for set_name, feature_cols in feature_sets.items():
    n_features = len(feature_cols)
    input_shape = (sequence_length, n_features)

    for model_name, builder_func in model_builders.items():
        cache_key = f"{model_name}_{set_name}"
        # Build model and save initial weights
        model = builder_func(input_shape)
        initial_weights = model.get_weights()

        model_templates[cache_key] = {
            'model': model,
            'initial_weights': initial_weights,
            'input_shape': input_shape
        }

print(f"Created {len(model_templates)} model templates.")
print(f"Expected total models: {len(ticker_list) * 12} = {len(ticker_list) * 12}")

# Training Loop
for ticker in tqdm(ticker_list, desc="Processing Tickers"):

    for set_name, feature_cols in feature_sets.items():

        data = prepare_data_for_ticker(
            ticker, df, feature_cols,
            sequence_length=sequence_length
        )

        if data is None:
            continue

        X_train = data['X_train']
        X_test = data['X_test']
        y_train = data['y_train']
        y_test = data['y_test']

        # Skip if unexpected shapes
        if len(X_test) < 5 or X_train.shape[1] != sequence_length:
            continue

        for model_name in model_builders.keys():
            try:
                cache_key = f"{model_name}_{set_name}"
                template = model_templates[cache_key]
                model = template['model']

                # Reset to initial weights
                model.set_weights(template['initial_weights'])

                # Train
                history = model.fit(
                    X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stop],
                    verbose=0,
                    validation_split=0.1
                )

                # Predict
                y_pred = model.predict(X_test, verbose=0).flatten()

                # Calculate metrics
                metrics = calculate_metrics(
                    y_test, y_pred,
                    model_name, ticker, set_name
                )

                if metrics:
                    all_results.append(metrics)

                del history

            except Exception as e:
                continue

    # Clear session every 10 tickers to prevent memory buildup
    if (ticker_list.index(ticker) + 1) % 10 == 0:
        tf.keras.backend.clear_session()

print(f"\nDeep Learning Training Complete. Total results: {len(all_results)}")
print(f"Expected: ~{len(ticker_list) * 12}, Actual: {len(all_results)}")

# Final cleanup
for template in model_templates.values():
    del template['model']
tf.keras.backend.clear_session()

In [ ]:
# Cell 8: Save Results
print("Saving results.")

if all_results:
    results_df = pd.DataFrame(all_results)
    results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

    csv_path = output_dir / "03B_Deep_Learning_Performance.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"Successfully saved {len(results_df)} results to {csv_path}")

    print("Saving results to 'results_deep_learning_models' table in database.")
    try:
        results_df.to_sql(
            'results_deep_learning_models',
            con=engine,
            if_exists='replace',
            index=False,
            chunksize=1000
        )
        print("Successfully wrote results to database!")
    except Exception as e:
        print(f"Error writing results to database: {e}")
else:
    print("No results were generated. Check for errors.")

In [ ]:
# Cell 10: Continue Training on Additional Tickers (Batch 2)
print("Continuing Deep Learning Training (Batch 2).")

# Load existing results to avoid retraining
existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
trained_tickers = set(existing_results_df['ticker'].unique())
print(f"Already trained on {len(trained_tickers)} tickers.")

# Get next batch of tickers
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]
print(f"Remaining tickers available: {len(remaining_tickers)}")

# Set batch size (start conservative)
BATCH_SIZE = 100  # Start with 100, can increase to 150-200 if stable
next_batch = remaining_tickers[:BATCH_SIZE]
print(f"Training on next {len(next_batch)} tickers.")

# Pre-filter for data availability
valid_tickers = []
for ticker in tqdm(next_batch, desc="Validating Tickers"):
    ticker_df = df[df['ticker'] == ticker]
    if len(ticker_df) >= sequence_length + 100:
        valid_tickers.append(ticker)

print(f"Valid tickers: {len(valid_tickers)}")

# Rebuild model templates (same as before)
print("\nRebuilding model templates.")
model_templates = {}

for set_name, feature_cols in feature_sets.items():
    n_features = len(feature_cols)
    input_shape = (sequence_length, n_features)

    for model_name, builder_func in model_builders.items():
        cache_key = f"{model_name}_{set_name}"
        model = builder_func(input_shape)
        initial_weights = model.get_weights()

        model_templates[cache_key] = {
            'model': model,
            'initial_weights': initial_weights,
            'input_shape': input_shape
        }

print(f"Created {len(model_templates)} model templates.")

# Training Loop
batch_results = []

for ticker in tqdm(valid_tickers, desc="Processing Additional Tickers"):

    for set_name, feature_cols in feature_sets.items():

        data = prepare_data_for_ticker(
            ticker, df, feature_cols,
            sequence_length=sequence_length
        )

        if data is None:
            continue

        X_train = data['X_train']
        X_test = data['X_test']
        y_train = data['y_train']
        y_test = data['y_test']

        if len(X_test) < 5 or X_train.shape[1] != sequence_length:
            continue

        for model_name in model_builders.keys():
            try:
                cache_key = f"{model_name}_{set_name}"
                template = model_templates[cache_key]
                model = template['model']

                model.set_weights(template['initial_weights'])

                history = model.fit(
                    X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stop],
                    verbose=0,
                    validation_split=0.1
                )

                y_pred = model.predict(X_test, verbose=0).flatten()

                metrics = calculate_metrics(
                    y_test, y_pred,
                    model_name, ticker, set_name
                )

                if metrics:
                    batch_results.append(metrics)

                del history

            except Exception as e:
                continue

    # Clear session every 10 tickers
    if (valid_tickers.index(ticker) + 1) % 10 == 0:
        tf.keras.backend.clear_session()

print(f"\nBatch 2 Training Complete. New results: {len(batch_results)}")

# Cleanup
for template in model_templates.values():
    del template['model']
tf.keras.backend.clear_session()

# Append to existing results
if batch_results:
    new_results_df = pd.DataFrame(batch_results)
    new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Combine with existing
    combined_df = pd.concat([existing_results_df, new_results_df], ignore_index=True)

    # Save updated CSV
    combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
    print(f"Updated results file. Total results: {len(combined_df)}")

    # Update database
    try:
        combined_df.to_sql(
            'results_deep_learning_models',
            con=engine,
            if_exists='replace',
            index=False,
            chunksize=1000
        )
        print("Successfully updated database!")
    except Exception as e:
        print(f"Database update error: {e}")
else:
    print("No new results generated.")

In [ ]:
# Cell: Fix Database Upload (Run this first)
import pandas as pd
from sqlalchemy import create_engine

print("Attempting to fix database connection and upload existing results...")

# 1. Force close the existing bad connection
try:
    engine.dispose()
    print("Old engine disposed.")
except:
    pass

# 2. Re-create the engine
db_url = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
engine = create_engine(db_url)
print("New database engine created.")

# 3. Load the CSV that was successfully saved on disk
try:
    csv_path = output_dir / "03B_Deep_Learning_Performance.csv"
    current_results = pd.read_csv(csv_path)
    print(f"Loaded {len(current_results)} rows from CSV.")

    # 4. Upload to Database
    current_results.to_sql(
        'results_deep_learning_models',
        con=engine,
        if_exists='replace',
        index=False,
        chunksize=1000
    )
    print("SUCCESS: All results successfully uploaded to the database.")

except Exception as e:
    print(f"CRITICAL ERROR: Could not upload to database. Error: {e}")

In [ ]:
# Cell: Train Batch 3 (Tickers 200-300)
print("Starting Batch 3 Training.")

# 1. Refresh the list of what is already finished
existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
trained_tickers = set(existing_results_df['ticker'].unique())
print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Calculate the next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

# Grab the next 100
BATCH_SIZE = 100
next_batch = remaining_tickers[:BATCH_SIZE]
print(f"Targeting next {len(next_batch)} tickers.")

# 3. Pre-filter for data availability
valid_tickers = []
for ticker in tqdm(next_batch, desc="Validating Tickers"):
    ticker_df = df[df['ticker'] == ticker]
    if len(ticker_df) >= sequence_length + 100:
        valid_tickers.append(ticker)

print(f"Valid tickers to train: {len(valid_tickers)}")

# 4. Rebuild Templates (Fresh start for memory safety)
tf.keras.backend.clear_session()
model_templates = {}
for set_name, feature_cols in feature_sets.items():
    input_shape = (sequence_length, len(feature_cols))
    for model_name, builder_func in model_builders.items():
        model = builder_func(input_shape)
        model_templates[f"{model_name}_{set_name}"] = {
            'model': model,
            'initial_weights': model.get_weights(),
            'input_shape': input_shape
        }

# 5. Training Loop
batch_results = []
for ticker in tqdm(valid_tickers, desc="Training Batch 3"):
    for set_name, feature_cols in feature_sets.items():
        data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
        if data is None: continue

        # Skip if validation set is too small
        if len(data['X_test']) < 5: continue

        for model_name in model_builders.keys():
            try:
                cache_key = f"{model_name}_{set_name}"
                template = model_templates[cache_key]
                model = template['model']
                model.set_weights(template['initial_weights']) # Reset weights

                history = model.fit(
                    data['X_train'], data['y_train'],
                    epochs=50, batch_size=32, callbacks=[early_stop],
                    verbose=0, validation_split=0.1
                )

                y_pred = model.predict(data['X_test'], verbose=0).flatten()
                metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)

                if metrics: batch_results.append(metrics)
                del history
            except: continue

    # Memory management
    if (valid_tickers.index(ticker) + 1) % 10 == 0:
        tf.keras.backend.clear_session()

# 6. Save and Upload
print(f"Batch 3 Complete. Generated {len(batch_results)} new results.")

if batch_results:
    # Append to CSV
    new_results_df = pd.DataFrame(batch_results)
    new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Reload full CSV to be safe, append, and save back
    full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
    combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
    print(f"CSV updated. Total rows: {len(combined_df)}")

    # Upload to DB
    try:
        # Re-dispose engine one last time to be safe
        engine.dispose()
        engine = create_engine(db_url)

        combined_df.to_sql(
            'results_deep_learning_models',
            con=engine,
            if_exists='replace',
            index=False,
            chunksize=1000
        )
        print("Database successfully updated with Batch 3.")
    except Exception as e:
        print(f"Error uploading Batch 3 to DB: {e}")

# Cleanup
for template in model_templates.values(): del template['model']
tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 4
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime

print("Starting Batch 4 Training...")

# 1. Refresh the list of what is already finished
# We reload the CSV to ensure we skip everything done in Batch 3
existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
trained_tickers = set(existing_results_df['ticker'].unique())
print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Calculate the next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("No tickers left to train!")
else:
    # Grab the next 100
    BATCH_SIZE = 100
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 4).")

    # 3. Pre-filter for data availability
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 4 Tickers"):
        ticker_df = df[df['ticker'] == ticker]
        if len(ticker_df) >= sequence_length + 100:
            valid_tickers.append(ticker)

    print(f"Valid tickers to train: {len(valid_tickers)}")

    # 4. Rebuild Templates (Clear session to prevent memory leaks)
    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    # 5. Training Loop
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 4"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None: continue

            if len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']
                    model.set_weights(template['initial_weights'])

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)

                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        # Periodic memory clear
        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    # 6. Save and Upload
    print(f"Batch 4 Complete. Generated {len(batch_results)} new results.")

    if batch_results:
        # Append new results to CSV
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Combine with full dataset
        full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
        combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
        combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
        print(f"CSV updated. Total rows: {len(combined_df)}")

        # Upload to DB (Re-initializing engine to prevent transaction errors)
        try:
            engine.dispose()
            engine = create_engine(db_url)

            combined_df.to_sql(
                'results_deep_learning_models',
                con=engine,
                if_exists='replace',
                index=False,
                chunksize=1000
            )
            print("Database successfully updated with Batch 4.")
        except Exception as e:
            print(f"Error uploading Batch 4 to DB: {e}")

    # Final Cleanup
    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Assess Performance (Current Progress ~400 tickers)
import pandas as pd
import numpy as np

print("Assessing Deep Learning Model Performance so far...")

# 1. Load Results from CSV
try:
    results_path = output_dir / "03B_Deep_Learning_Performance.csv"
    results_df = pd.read_csv(results_path)
    print(f"Loaded {len(results_df)} results from {results_path}")
    print(f"Unique tickers processed: {results_df['ticker'].nunique()}")
except Exception as e:
    print(f"Error loading results: {e}")
    results_df = pd.DataFrame()

if not results_df.empty:
    # 2. Summary Statistics by Model Type
    # We group by model to see which architecture performs best on average
    summary = results_df.groupby('model').agg({
        'rmse': ['mean', 'std', 'min'],
        'mae': ['mean', 'std'],
        'directional_accuracy': ['mean', 'std', 'max']
    }).round(4)

    print("\n--- Performance by Model Type ---")
    print(summary.to_string())

    # 3. Top 15 Models by RMSE
    print("\n--- Top 15 Models by RMSE ---")
    top_models = results_df.nsmallest(15, 'rmse')[['model', 'ticker', 'rmse', 'directional_accuracy']]
    print(top_models.to_string(index=False))

    # 4. Comparison with Traditional Models (if available in DB)
    try:
        trad_results = pd.read_sql("SELECT * FROM results_ir_models", con=engine)

        print("\n--- Comparison with Traditional Models ---")
        print(f"{'Metric':<25} | {'Deep Learning':<15} | {'Traditional':<15}")
        print("-" * 60)

        # RMSE Comparison
        dl_min_rmse = results_df['rmse'].min()
        trad_min_rmse = trad_results['rmse'].min()
        print(f"{'Best RMSE':<25} | {dl_min_rmse:<15.4f} | {trad_min_rmse:<15.4f}")

        # Directional Accuracy Comparison
        dl_avg_acc = results_df['directional_accuracy'].mean()
        trad_avg_acc = trad_results['directional_accuracy'].mean()
        print(f"{'Avg Directional Acc %':<25} | {dl_avg_acc:<15.2f} | {trad_avg_acc:<15.2f}")

        dl_best_acc = results_df['directional_accuracy'].max()
        trad_best_acc = trad_results['directional_accuracy'].max()
        print(f"{'Best Directional Acc %':<25} | {dl_best_acc:<15.2f} | {trad_best_acc:<15.2f}")

    except Exception as e:
        print(f"\n(Could not load traditional model results for comparison: {e})")

else:
    print("No results found to analyze.")

In [ ]:
# Cell: Train Batch 5 (Next 200 tickers -> Target Total ~600)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime

sequence_length = 60
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)
print("Starting Batch 5 Training (Targeting next 200 tickers).")

# 1. Identify what is already done
try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except:
    trained_tickers = set()
    existing_results_df = pd.DataFrame()

print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Select next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers have been trained!")
else:
    BATCH_SIZE = 200  # Training 200 tickers in this batch
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers.")

    # 3. Validate Data Availability
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Tickers"):
        ticker_df = df[df['ticker'] == ticker]
        # Check if enough data exists (Sequence length + buffer)
        if len(ticker_df) >= sequence_length + 100:
            valid_tickers.append(ticker)

    print(f"Valid tickers to train: {len(valid_tickers)}")

    # 4. Rebuild Model Templates (Fresh session)
    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    # 5. Training Loop
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 5"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None: continue
            if len(data['X_test']) < 5: continue # Skip small test sets

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']
                    model.set_weights(template['initial_weights']) # Reset

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)

                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        # Memory Cleanup (every 10 tickers)
        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    # 6. Save Results
    print(f"Batch 5 Complete. Generated {len(batch_results)} new results.")

    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Merge and Save CSV
        full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
        combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
        combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
        print(f"CSV updated. Total rows: {len(combined_df)}")

        # Upload to DB (Re-init engine to act as a fresh transaction)
        try:
            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql(
                'results_deep_learning_models',
                con=engine,
                if_exists='replace',
                index=False,
                chunksize=1000
            )
            print("Database successfully updated.")
        except Exception as e:
            print(f"Error uploading to DB: {e}")

    # Final Cleanup
    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 6 (Next 200 tickers)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
sequence_length = 60
BATCH_SIZE = 200
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 6 Training (Targeting next 200 tickers).")

# 1. Identify what is already done
try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except Exception as e:
    print(f"Could not load existing results: {e}")
    trained_tickers = set()
    existing_results_df = pd.DataFrame()

print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Select next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers have been trained! No more batches needed.")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 6).")

    # 3. Validate Data Availability
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 6 Tickers"):
        ticker_df = df[df['ticker'] == ticker]
        if len(ticker_df) >= sequence_length + 100:
            valid_tickers.append(ticker)

    print(f"Valid tickers to train: {len(valid_tickers)}")

    # 4. Rebuild Model Templates (Fresh session to clear memory)
    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    # 5. Training Loop
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 6"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None: continue
            if len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']

                    # Reset weights to initial state
                    model.set_weights(template['initial_weights'])

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)

                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        # Memory Cleanup (every 10 tickers)
        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    # 6. Save Results
    print(f"Batch 6 Complete. Generated {len(batch_results)} new results.")

    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Merge and Save CSV
        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")

            # Upload to DB (Re-init engine to act as a fresh transaction)
            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql(
                'results_deep_learning_models',
                con=engine,
                if_exists='replace',
                index=False,
                chunksize=1000
            )
            print("Database successfully updated.")
        except Exception as e:
            print(f"Error saving/uploading results: {e}")

    # Final Cleanup
    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 7 (Tickers 800-1000)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
sequence_length = 60
BATCH_SIZE = 200  # Target tickers 800-1000
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 7 Training (Targeting next 200 tickers).")

# 1. Identify what is already done
try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except Exception as e:
    print(f"Could not load existing results: {e}")
    trained_tickers = set()
    existing_results_df = pd.DataFrame()

print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Select next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers have been trained! No more batches needed.")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 7).")

    # 3. Validate Data Availability
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 7 Tickers"):
        ticker_df = df[df['ticker'] == ticker]
        if len(ticker_df) >= sequence_length + 100:
            valid_tickers.append(ticker)

    print(f"Valid tickers to train: {len(valid_tickers)}")

    # 4. Rebuild Model Templates (Fresh session to clear memory)
    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    # 5. Training Loop
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 7"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None: continue
            if len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']

                    # Reset weights to initial state
                    model.set_weights(template['initial_weights'])

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)

                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        # Memory Cleanup (every 10 tickers)
        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    # 6. Save Results
    print(f"Batch 7 Complete. Generated {len(batch_results)} new results.")

    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Merge and Save CSV
        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")

            # Upload to DB (Re-init engine to act as a fresh transaction)
            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql(
                'results_deep_learning_models',
                con=engine,
                if_exists='replace',
                index=False,
                chunksize=1000
            )
            print("Database successfully updated.")
        except Exception as e:
            print(f"Error saving/uploading results: {e}")

    # Final Cleanup
    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 8 (Tickers 1000-1200)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
sequence_length = 60
BATCH_SIZE = 200  # Target next 200 tickers
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 8 Training (Targeting next 200 tickers).")

# 1. Identify what is already done
try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except Exception as e:
    print(f"Could not load existing results: {e}")
    trained_tickers = set()
    existing_results_df = pd.DataFrame()

print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Select next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers have been trained! No more batches needed.")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 8).")

    # 3. Validate Data Availability
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 8 Tickers"):
        ticker_df = df[df['ticker'] == ticker]
        if len(ticker_df) >= sequence_length + 100:
            valid_tickers.append(ticker)

    print(f"Valid tickers to train: {len(valid_tickers)}")

    # 4. Rebuild Model Templates (Fresh session to clear memory)
    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    # 5. Training Loop
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 8"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None: continue
            if len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']

                    # Reset weights to initial state
                    model.set_weights(template['initial_weights'])

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)

                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        # Memory Cleanup (every 10 tickers)
        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    # 6. Save Results
    print(f"Batch 8 Complete. Generated {len(batch_results)} new results.")

    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Merge and Save CSV
        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")

            # Upload to DB (Re-init engine to act as a fresh transaction)
            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql(
                'results_deep_learning_models',
                con=engine,
                if_exists='replace',
                index=False,
                chunksize=1000
            )
            print("Database successfully updated.")
        except Exception as e:
            print(f"Error saving/uploading results: {e}")

    # Final Cleanup
    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 9 (Tickers 1200-1400)
import pandas as pd
import gc
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
sequence_length = 60
BATCH_SIZE = 200
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 9 Training (Targeting next 200 tickers)")

# 1. Identify what is already done
try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except Exception as e:
    trained_tickers = set()

print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Select next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers have been trained!")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 9).")

    # 3. Validate Data
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 9 Tickers"):
        if len(df[df['ticker'] == ticker]) >= sequence_length + 100:
            valid_tickers.append(ticker)
    print(f"Valid tickers: {len(valid_tickers)}")

    # 4. Prepare Weights (Store weights only, not model objects)
    tf.keras.backend.clear_session()
    weight_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            # Build temp model just to get initial random weights
            temp_model = builder_func(input_shape)
            weight_templates[f"{model_name}_{set_name}"] = {
                'weights': temp_model.get_weights(),
                'input_shape': input_shape,
                'builder': builder_func
            }
            del temp_model
    tf.keras.backend.clear_session()

    # 5. Training Loop (Constructing FRESH models per ticker to stop retracing)
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 9"):

        # Prepare data
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None or len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = weight_templates[cache_key]

                    # FIX: Build a FRESH model instance every time
                    # This prevents TensorFlow from trying to reuse the graph for different X_test sizes
                    model = template['builder'](template['input_shape'])
                    model.set_weights(template['weights'])

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)
                    if metrics: batch_results.append(metrics)

                    # Aggressive cleanup
                    del model
                    del history
                except Exception as e:
                    continue

        # Clear session after EVERY ticker to prevent memory bloat from fresh models
        tf.keras.backend.clear_session()
        gc.collect()

    # 6. Save Results
    print(f"Batch 9 Complete. Generated {len(batch_results)} new results.")
    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')
        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")

            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql('results_deep_learning_models', con=engine, if_exists='replace', index=False, chunksize=1000)
            print("Database updated.")
        except Exception as e: print(f"Error: {e}")

In [ ]:
# Cell: Train Batch 10 (Tickers 1400-1600)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
sequence_length = 60
BATCH_SIZE = 200
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 10 Training (Targeting next 200 tickers).")

# 1. Identify what is already done
try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except Exception as e:
    print(f"Could not load existing results: {e}")
    trained_tickers = set()
    existing_results_df = pd.DataFrame()

print(f"Currently trained on {len(trained_tickers)} tickers.")

# 2. Select next batch
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers have been trained! No more batches needed.")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 10).")

    # 3. Validate Data Availability
    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 10 Tickers"):
        ticker_df = df[df['ticker'] == ticker]
        if len(ticker_df) >= sequence_length + 100:
            valid_tickers.append(ticker)

    print(f"Valid tickers to train: {len(valid_tickers)}")

    # 4. Rebuild Model Templates
    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    # 5. Training Loop
    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 10"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None: continue
            if len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']
                    model.set_weights(template['initial_weights'])

                    history = model.fit(
                        data['X_train'], data['y_train'],
                        epochs=50, batch_size=32, callbacks=[early_stop],
                        verbose=0, validation_split=0.1
                    )

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)
                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    # 6. Save Results
    print(f"Batch 10 Complete. Generated {len(batch_results)} new results.")
    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')

        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")

            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql('results_deep_learning_models', con=engine, if_exists='replace', index=False, chunksize=1000)
            print("Database successfully updated.")
        except Exception as e:
            print(f"Error saving results: {e}")

    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 11 (Tickers 1600-1800)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

sequence_length = 60
BATCH_SIZE = 200
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 11 Training (Targeting next 200 tickers).")

try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except:
    trained_tickers = set()

print(f"Currently trained on {len(trained_tickers)} tickers.")
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers trained.")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 11).")

    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 11 Tickers"):
        if len(df[df['ticker'] == ticker]) >= sequence_length + 100:
            valid_tickers.append(ticker)
    print(f"Valid tickers: {len(valid_tickers)}")

    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 11"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None or len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']
                    model.set_weights(template['initial_weights'])

                    history = model.fit(data['X_train'], data['y_train'], epochs=50, batch_size=32,
                                      callbacks=[early_stop], verbose=0, validation_split=0.1)

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)
                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    print(f"Batch 11 Complete. New results: {len(batch_results)}")
    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')
        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")
            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql('results_deep_learning_models', con=engine, if_exists='replace', index=False, chunksize=1000)
            print("Database updated.")
        except Exception as e: print(f"Error: {e}")

    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()

In [ ]:
# Cell: Train Batch 12 (Tickers 1800-2000)
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf
from tqdm.notebook import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

sequence_length = 60
BATCH_SIZE = 200
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

print("Starting Batch 12 Training (Targeting next 200 tickers).")

try:
    existing_results_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
    trained_tickers = set(existing_results_df['ticker'].unique())
except:
    trained_tickers = set()

print(f"Currently trained on {len(trained_tickers)} tickers.")
all_tickers = sorted(df['ticker'].unique())
remaining_tickers = [t for t in all_tickers if t not in trained_tickers]

if not remaining_tickers:
    print("All tickers trained.")
else:
    next_batch = remaining_tickers[:BATCH_SIZE]
    print(f"Targeting next {len(next_batch)} tickers (Batch 12).")

    valid_tickers = []
    for ticker in tqdm(next_batch, desc="Validating Batch 12 Tickers"):
        if len(df[df['ticker'] == ticker]) >= sequence_length + 100:
            valid_tickers.append(ticker)
    print(f"Valid tickers: {len(valid_tickers)}")

    tf.keras.backend.clear_session()
    model_templates = {}
    for set_name, feature_cols in feature_sets.items():
        input_shape = (sequence_length, len(feature_cols))
        for model_name, builder_func in model_builders.items():
            model = builder_func(input_shape)
            model_templates[f"{model_name}_{set_name}"] = {
                'model': model,
                'initial_weights': model.get_weights(),
                'input_shape': input_shape
            }

    batch_results = []
    for ticker in tqdm(valid_tickers, desc="Training Batch 12"):
        for set_name, feature_cols in feature_sets.items():
            data = prepare_data_for_ticker(ticker, df, feature_cols, sequence_length=sequence_length)
            if data is None or len(data['X_test']) < 5: continue

            for model_name in model_builders.keys():
                try:
                    cache_key = f"{model_name}_{set_name}"
                    template = model_templates[cache_key]
                    model = template['model']
                    model.set_weights(template['initial_weights'])

                    history = model.fit(data['X_train'], data['y_train'], epochs=50, batch_size=32,
                                      callbacks=[early_stop], verbose=0, validation_split=0.1)

                    y_pred = model.predict(data['X_test'], verbose=0).flatten()
                    metrics = calculate_metrics(data['y_test'], y_pred, model_name, ticker, set_name)
                    if metrics: batch_results.append(metrics)
                    del history
                except: continue

        if (valid_tickers.index(ticker) + 1) % 10 == 0:
            tf.keras.backend.clear_session()

    print(f"Batch 12 Complete. New results: {len(batch_results)}")
    if batch_results:
        new_results_df = pd.DataFrame(batch_results)
        new_results_df['timestamp'] = datetime.now().strftime('%Y%m%d_%H%M%S')
        try:
            full_df = pd.read_csv(output_dir / "03B_Deep_Learning_Performance.csv")
            combined_df = pd.concat([full_df, new_results_df], ignore_index=True)
            combined_df.to_csv(output_dir / "03B_Deep_Learning_Performance.csv", index=False)
            print(f"CSV updated. Total rows: {len(combined_df)}")
            engine.dispose()
            engine = create_engine(db_url)
            combined_df.to_sql('results_deep_learning_models', con=engine, if_exists='replace', index=False, chunksize=1000)
            print("Database updated.")
        except Exception as e: print(f"Error: {e}")

    for template in model_templates.values(): del template['model']
    tf.keras.backend.clear_session()